# k-Nearest Neighbour (kNN) Classification

The k-nearest neighbour method is conceptually one of the easiest classification models to understand. In fact, it is so simple that just describing the algorithm should suffice to understand it.

The starting point is a set of $M$ training datapoints, or, better said, reference datapoints $x^R$. Note that each point is characterised by the value of its $N$ independent components $x_i^T$ (i=1,N), together with the corresponding label $Y$. Given this reference and a new point $x^T$ whose value of $Y$ we want to predict, we do the following: 

1) Measure the "distance" between the point $x^T$ and each of the reference datapoints. The distance can be for example the Euclidean distance $d=\sum_i (x^T_i - x^R_i)^2$, but other definitions of [distance](https://en.wikipedia.org/wiki/Distance#Mathematics) can be used if more appropriate. For example, if the input data are not continuous variables but have binary values, the [Hamming distance](https://en.wikipedia.org/wiki/Hamming_distance) might be a good choice.

2) Take the $k$ datapoints that are closest to our test point, where $k$ can be any integer (but lower than $M$, for obvious reasons). $k$ is an hyper-parameter of a kNN model.

3) The predicted class is the mode of these $k$ points - which is the class with the most occurences within the k nearest datapoints. 

Below we present some code to show how this algorithm works if one were to calculate it manually.

> Here we revisit the use of Python classes in the `KNearestNeighbour` class (defined at the end) - in reality it is quicker and more reliable to use the `scikit-learn` package since this has been optimised to run as quickly as possible

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Manually calculating the k-nearest neighbours

To build a k-nearest neighbours algorithm, we will take the following steps:

1. Measure the distance between each point in `X_test` and every single point in `X_train`
2. Get the arguments of the k smallest values
3. Map the indices from the k smallest values to the classes contained in `y_train`
4. For all of the subsets of classes, pick the most common occurence and this is the prediction to be appended to `y_test`

Few of these steps are simple to program manually in an efficient way, so we will make use of some of the more advanced functions available in the `collections` module, as well as `numpy` and `scipy`.

### Step 1 - Measuring the distance between two arrays - `scipy.spatial.distance.cdist`

You should remember the function `scipy.spatial.distance.cdist`, which we discussed in a previous lecture. It can be imported by writing:

```python
from scipy.spatial.distance import cdist
```

This function takes two series of arrays, $A_m$ and $B_n$. All these arrays, regardless if they come from the $A$ or $B$ series, has the same dimensionality, whereas the number of arrays in each series might differ. In the example below these two series are called `X_test` ($\equiv A$) and `X_train` ($\equiv B$) respectively, and form the arguments of the function `get_k_nearest_neighbours`. The result of calling this function is an array that is allocated to the variable `distances`, which follows this structure (where `x->y` is the distance between `x` and `y`):

```python
[
    [A1->B1, A1->B2, A1->B3, ..., A1->Bn],
    [A2->B1, A2->B2, A2->B3, ..., A2->Bn],
    [  ... ,   ... ,   ... , ...,   ... ],
    [Am->B1, Am->B2, Am->B3, ..., Am->Bn],
]
```

### Step 2 - Get the indices of the k smallest values

To simlpify the code and allow it to be more readable, we loop over all the rows in the distance matrix, where each row is all of the distances between one point in `X_test` and all of the points in `X_train`. In the schematic above, this is the row that looks like `[A1->B1, ... A1->Bn]`. A single line of code allows us to select the k-smallest values.

```python
np.argsort(row)[:k]
```

The first part `np.argsort(row)` returns an array of the indices of the `row` in ascending order. By slicing the array using `[:k]`, we select the first `k` of these indices, and then repeat for all rows.

In [2]:
from scipy.spatial.distance import cdist

def get_k_nearest_neighbours(X_test: np.ndarray, X_train: np.ndarray, k: int = 3) -> list:
    """Returns the indices from array B that represent the k nearest neighbours"""
    
    # get the distances
    distances = cdist(X_test, X_train)
    
    # initialise list for returning
    k_nearest_neighbours = []
    
    # loop over all rows
    for row in distances:
        
        # use np.argsort to get k-lowest indices
        indices = np.argsort(row)[:k]
        
        # append to final list
        k_nearest_neighbours.append(indices)
        
    return k_nearest_neighbours
    

## Creating a class that is similar to `scikit-learn`

### Step 3 - Map the indices to the classes

Within the `KNearestNeighbour.predict` method, we access the previously written function `get_k_nearest_neighbours` which returns the indices of the nearest neighbours. It is now necessary to map these to the classes contained in `self.y_train`. 

For example if a set of indices was `[1, 0, 4]` and `self.y_train` was `['A', 'B', 'A', 'C', 'B', 'C', 'A', 'B']`, then the result of the mapping would be `['B', 'A', 'B']` because these are the 2nd, 1st and 5th elements in `self.y_train`. 

To do this, we run the following code, to get a list of classes - for example from `[[6, 7, 0], [4, 1, 7]]` to `[['A', 'B', 'A'], ['B', 'B', 'B']]`

```python
indices_list = get_k_nearest_neighbours(X_test, self.X_train, k=k)
classes_list = []
for indices in indices_list:
    classes_list.append([y_train[i] for i in indices])
```

### Step 4 - Pick the most common occurence for each row in `X_test`

In this section, we use `Counter(row).most_common(1)` to get the most common occurence in a row. The function takes a list such as `['A', 'A', 'B']` and returns a list with a tuple containing two elements: the most common element and the number it occurs in the input. To make a specific example:

```python
>>> Counter(['A', 'A', 'B']).most_common(1)
[('A', 2)]
```

Since we only care about the class itself, not the number of occurences, we can index the result by doing `Counter(row).most_common(1)[0][0]` to select the first element of the first element in the resulting list.

In [3]:
from collections import Counter

class KNearestNeighbour:
    """Manual implementation of the k-Nearest Neighbour classifier"""
    def __init__(self, X_train: np.ndarray, y_train: np.ndarray):
        self.X_train = X_train
        self.y_train = y_train
        self.classes = set(self.y_train)
        
    def predict(self, X_test: np.ndarray, k: int = 3) -> np.ndarray:
        """Returns a 1D array containing predicted classification values"""
        # use previously defined function to get indices of most common nearest neighbours
        indices_list = get_k_nearest_neighbours(X_test, self.X_train, k=k)
        
        # convert indices (currently a list of numbers) to a list of classes
        classes_list = []
        for indices in indices_list:
            classes_list.append([y_train[i] for i in indices])
        
        # use Counter(list).most_common to retrieve most common occurences
        y_pred = []
        for classes in classes_list:
            most_common = Counter(classes).most_common(1)[0][0]
            result.append(most_common)
        
        return np.array(y_pred)
    
    def score(self, X_test: np.ndarray, y_test: np.ndarray) -> np.ndarray:
        """Returns a score by comparing predicted values to real test values"""
        
        # get the predicted values using self.predict
        y_pred = self.predict(X_test)
        
        # measure the number of correct values
        number_correct = 0
        for i, predicted_value in enumerate(y_pred):
            test_value = y_test[i]
            if test_value == predicted_value:
                number_correct += 1
        
        # take ratio compared to maximum number of values
        score = number_correct / len(y_test)
        
        return score

## Quick Exercise

Write a fake reference dataset with a few points and check that the above algorithm provides the correct answer by testing it on some arbitrary test values. Typically, you would do that by using a simple dataset, for which the answer can be computed by hand!

In [ ]:
### Write your simple code here

## Sci-kit learn

Doing all of the above in sci-kit learn, as usual, is easy and only requires a few lines of code. 

You should be having a look at [this page](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) for the exact syntax, since **`sci-kit learn` allows you a much greater control on the parameters that you can use, as well as on the algorithm to compute the nearest neighbours**. 

In fact, you should notice that what we implemented above is a brute force algorithm but smarter approaches are possible. For example, we calculated the distance between the test point and *each* point in the reference dataset, although most of them will be irrelevant! This is a problem in particular when the number of points in the reference set is very large, something which is not that uncommon. Moreover, whereas we have chosen the mode of the k-nearest neighbour to decide our final value, other choices are possible.

Here we just show you a snippet of pseudo-code equivalent to the previous one shown above:

`
from sklearn.neighbors import KNeighborsClassifier #Import the necessary class
neigh = KNeighborsClassifier(n_neighbors=3) #Define a kNN using 3 nearest neighbours
neigh.fit(X, y) #Train the kNN on the training dataset X,y
neigh.predict([[XT]]) # Output the predicted class for XT
`

You should be carefully reading the exact syntax of `KNeighborsClassifier` to see the exact form of `X` and `y` required! You will have the possibility to experiment with it during tasks and exercises.

## Food for thought 

A decision tree can be seen as a sort of kNN algorithm. How would you use it for classification? **Similarly, a kNN can be used for regression**. Can you think about a possible way to do it?